# Installation et imports

### Installation des modules 

In [4]:
%pip install Pandas


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install Matplotlib 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install Scikit-learn


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install Numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import et données

In [8]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import math


In [9]:
data = pd.read_csv("../dfClean.csv")

counts = data['descr_grav'].value_counts()

total_0 = counts[0]/len(data)
total_1 = counts[1]/len(data)
total_2 = counts[2]/len(data)
total_3 = counts[3]/len(data)

Proportion_0 = data[data['descr_grav'] == 0].sample(n=int(7500*total_0))
Proportion_1 = data[data['descr_grav'] == 1].sample(n=int(7500*total_1))
Proportion_2 = data[data['descr_grav'] == 2].sample(n=int(7500*total_2))
Proportion_3 = data[data['descr_grav'] == 3].sample(n=int(7500*total_3))

data_reduit = pd.concat([Proportion_0,Proportion_1, Proportion_2, Proportion_3])
data_reduit = data_reduit.reset_index(drop=True)
print(len(data_reduit))
display(data_reduit)

7498


,date,latitude,longitude,descr_cat_veh,descr_agglo,place,descr_dispo_secu,descr_grav
0,14026,45.6333,-1.083330,0,0,0,14,0
1,6248,48.8584,2.379500,0,0,0,14,0
2,19854,48.6000,7.600000,1,1,0,14,0
3,9077,43.6000,1.433330,0,0,0,14,0
4,7068,50.0500,1.416670,0,0,0,14,0
...,...,...,...,...,...,...,...,...
7493,13198,49.3170,0.269444,0,1,0,14,3
7494,28397,48.3000,4.050000,0,1,0,14,3
7495,8130,45.8167,-0.350000,0,1,0,14,3
7496,22952,48.2000,4.133330,0,1,0,14,3


# Répartition des données

### Répartition holdout 

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
def holdout(dataframe, colonne_predict, num_rep, train_s, test_s ):
        liste_data_train_test = []
        liste_data = []
        clean_data = dataframe.drop(colonne_predict, axis = 1)
        y = dataframe[colonne_predict]
        for i in range(num_rep):
                X_train, X_test, y_train, y_test = train_test_split(clean_data,y, train_size= train_s, test_size = test_s)
                X_train = X_train.reset_index(drop=True)
                y_train = y_train.reset_index(drop=True)
                X_test = X_test.reset_index(drop=True)
                y_test = y_test.reset_index(drop=True)
                liste_data_train_test.append([X_train, X_test, y_train, y_test])
        return liste_data_train_test

### Répartition Leave One Out

In [12]:
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

In [13]:
def leaveoneout(dataframe, colonne_predict,k):
    liste_y_pred = []
    liste_y_test = []
    X = dataframe.drop(colonne_predict, axis = 1) 
    y = dataframe[colonne_predict]
    LOO = LeaveOneOut()
    knn = KNeighborsClassifier(n_neighbors=k)
    
    for train_index, test_index in (LOO.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        liste_y_pred.append(y_pred)
        liste_y_test.append(y_test)
        
    return liste_y_pred, liste_y_test

# Classification avec KNN

### Fonctions from scratch

In [14]:
#Fonction de classification knn
def KnnClassification_distance_euclidienne(ligne,y_train,df,k):
    dictList = [None] * len(df)
    liste_y_pred = []
    for index, row in df.iterrows():
        dictList[index] = np.linalg.norm(ligne - row) #calcul de la distance euclidienne
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    for index,value in sorted_values:
        #firstk.append([df.loc[index]])
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

In [15]:
#Fonction de classification knn
def KnnClassification_distance_manhattan(ligne, y_train, df, k):
    dictList = [None] * len(df)
    liste_y_pred = []
    
    for index, row in df.iterrows():
        dictList[index] = np.sum(np.abs(ligne - row))  # Calcul de la distance de Manhattan
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    
    for index, value in sorted_values:
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

In [16]:
def KnnClassification_distance_sup_norme(ligne, y_train, df, k):
    dictList = [None] * len(df)
    liste_y_pred = []
    
    for index, row in df.iterrows():
        dictList[index] = np.max(np.abs(ligne - row))  # Calcul de la distance de la norme supérieure
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    
    for index, value in sorted_values:
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

### Scikit learn

In [17]:
#prediction holdout sklearn
def  KnnClassification_sk_learn(liste_data):
    knn = KNeighborsClassifier(n_neighbors=5)
    liste_y_pred_holdout_sk = []
    liste_y_test_holdout_sk = []
    for i in range(len(liste_data)):
        X_train = liste_data[i][0]
        X_test = liste_data[i][1]
        y_test = liste_data[i][3]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        liste_y_pred_holdout_sk.append(y_pred)
        liste_y_test_holdout_sk.append(y_test)
    return liste_y_pred_holdout_sk, liste_y_test_holdout_sk

### Calcul de score

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
#Répartition leave One Out
y_pred_LOO_5, y_test_LOO_5 = leaveoneout(data_reduit, "descr_grav", 5)
y_pred_LOO_7, y_test_LOO_7 = leaveoneout(data_reduit, "descr_grav", 7)
y_pred_LOO_10, y_test_LOO_10 = leaveoneout(data_reduit, "descr_grav", 10)

In [ ]:
# Score répartition leave One Out
accuracy_LOO_5 = accuracy_score(y_test_LOO_5, y_pred_LOO_5)
accuracy_LOO_7 = accuracy_score(y_test_LOO_7, y_pred_LOO_7)
accuracy_LOO_10 = accuracy_score(y_test_LOO_10, y_pred_LOO_10)

print(accuracy_LOO_5*100)
print(accuracy_LOO_7*100)
print(accuracy_LOO_10*100)

NameError: name 'y_test_LOO_5' is not defined

In [ ]:
#Répartition hold out
liste_data_holdout = []
#data_test = data.drop(columns=["latitude", "longitude"])
data_test = data_reduit
liste_data_holdout  = holdout(data_reduit,"descr_grav", 5, 0.8, 0.2)

# Prediction holdout from scratch
liste_y_pred_holdout_LOO_euclidienne = []
liste_y_pred_holdout_LOO_manhattan = []
liste_y_pred_holdout_LOO_sup_norme = []


X_test_0 = liste_data_holdout[0][1]
y_train_0  = liste_data_holdout[0][2]
X_train_0 = liste_data_holdout[0][0]
y_test_0 =  liste_data_holdout[0][3]

for i in range(len(X_test)):
    y_pred_holdout_sup_norme = KnnClassification_distance_sup_norme(X_test.loc[i], y_train, X_train, 5)
    liste_y_pred_holdout_LOO_sup_norme.append(y_pred_holdout_sup_norme)
    y_pred_holdout_euclidienne = KnnClassification_distance_euclidienne(X_test.loc[i], y_train, X_train, 5)
    liste_y_pred_holdout_LOO_euclidienne.append(y_pred_holdout_euclidienne)
    y_pred_holdout_manhattan = KnnClassification_distance_manhattan(X_test.loc[i], y_train, X_train, 5)
    liste_y_pred_holdout_LOO_manhattan.append(y_pred_holdout_manhattan)

for i in range(len(X_test)):
    y_pred_holdout_sup_norme = KnnClassification_distance_sup_norme(X_test.loc[i], y_train, X_train, 7)
    liste_y_pred_holdout_LOO_sup_norme.append(y_pred_holdout_sup_norme)
    y_pred_holdout_euclidienne = KnnClassification_distance_euclidienne(X_test.loc[i], y_train, X_train, 7)
    liste_y_pred_holdout_LOO_euclidienne.append(y_pred_holdout_euclidienne)
    y_pred_holdout_manhattan = KnnClassification_distance_manhattan(X_test.loc[i], y_train, X_train, 7)
    liste_y_pred_holdout_LOO_manhattan.append(y_pred_holdout_manhattan)

for i in range(len(X_test)):
    y_pred_holdout_sup_norme = KnnClassification_distance_sup_norme(X_test.loc[i], y_train, X_train, 10)
    liste_y_pred_holdout_LOO_sup_norme.append(y_pred_holdout_sup_norme)
    y_pred_holdout_euclidienne = KnnClassification_distance_euclidienne(X_test.loc[i], y_train, X_train, 10)
    liste_y_pred_holdout_LOO_euclidienne.append(y_pred_holdout_euclidienne)
    y_pred_holdout_manhattan = KnnClassification_distance_manhattan(X_test.loc[i], y_train, X_train, 10)
    liste_y_pred_holdout_LOO_manhattan.append(y_pred_holdout_manhattan)


NameError: name 'X_test' is not defined

In [ ]:
# Score répartition leave One Out
accuracy_holdout_euclidienne = accuracy_score(y_test_0, liste_y_pred_holdout_LOO_euclidienne)
accuracy_holdout_manhattan = accuracy_score(y_test_0, liste_y_pred_holdout_LOO_manhattan)
accuracy_holdout_sup_norme = accuracy_score(y_test_0, liste_y_pred_holdout_LOO_sup_norme)

print("Score euclidienne:" accuracy_holdout_euclidienne*100)
print("Score manhattan:" accuracy_holdout_manhattan*100)
print("Score sup norme:" accuracy_holdout_sup_norme*100)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (149622622.py, line 6)

In [ ]:
#Score prediction holdout scikit learn
#for i in range(len(liste_y_pred_holdout_sk)):
liste_y_pred_holdout_sk, liste_y_test_holdout_sk = KnnClassification_sk_learn(liste_data_holdout)
accuracy_holdout_sk = accuracy_score(liste_y_test_holdout_sk, liste_y_pred_holdout_sk)

# Classification avec trois algo "haut niveau"

## support vector machine (SVM)

step1

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

df = pd.read_csv("../dfClean.csv")


# Step 2: Sample the specified percentage of rows
df = df.sample(frac=0.06, random_state=42)

# Step 3: Store the sampled portion in a new DataFrame
df.reset_index(drop=True, inplace=True)  # Reset index if needed


# Step 1: Select feature columns (X) and target column (y)
feature_columns = df.columns.drop('descr_grav')
target_column = 'descr_grav'

X = df[feature_columns]
y = df[target_column]

step 2

In [9]:

# Step 2: Define the parameter grid
param_grid = {
    'C': [0.1, 1,10],
    'kernel': ['rbf'],
    'gamma': [0.1, 0.5, 1]
}


step 3 & 4

In [10]:

# Step 3: Create an SVM classifier
svm = SVC()

# Step 4: Create a GridSearchCV object
grid_search = GridSearchCV(svm, param_grid, scoring='accuracy')



step 5

In [16]:

# Step 5: Fit the GridSearchCV object
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=16)
grid_search.fit(X_train, y_train)



GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 0.5, 1],
                         'kernel': ['rbf']},
             scoring='accuracy')

step 6

In [43]:
# Step 6: Retrieve the best parameters and best score
best_params_svm = grid_search.best_params_
best_score = grid_search.best_score_
best_model = grid_search.best_estimator_
print("Best Parameters:", best_params_svm)
print("Best Score:", best_score)
ypred = best_model.predict(X_test)
print(confusion_matrix(y_test,ypred))


Best Parameters: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Best Score: 0.4054902922542408
[[372   0   0   0]
 [307   0   0   0]
 [167   0   0   0]
 [ 29   0   0   0]]


step 7

In [40]:

# Step 7: Train a new SVM model with the best parameters
best_svm = SVC(**best_params_svm)
best_svm.fit(X_train, y_train)



SVC(C=0.1, gamma=0.1)

In [30]:
import joblib

joblib.dump(best_svm,"../models/modelSVC.pkl")

['../models/modelSVC.pkl']

step 8

In [28]:
# Step 8: Evaluate the model on the test data or perform further analysis
accuracy = best_svm.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.42514285714285716


Random Forest

In [48]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

# Step 1: Select feature columns (X) and target column (y)
feature_columns = df.columns.drop('descr_grav')
target_column = 'descr_grav'

X = df[feature_columns]
y = df[target_column]

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=16)

# Step 3: Define the RandomForest classifier
rf = RandomForestClassifier()

# Step 4: Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Step 5: Create a GridSearchCV object
grid_search = GridSearchCV(rf, param_grid, scoring='accuracy')

# Step 6: Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Step 7: Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Step 8: Train the RandomForest model with the best parameters
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)

# Step 9: Evaluate the model on the testing data
accuracy = best_rf.score(X_test, y_test)




In [49]:
# Print the best parameters, best score, and accuracy
print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
best_model = grid_search.best_estimator_
ypred = best_model.predict(X_test)
print(confusion_matrix(y_test,ypred))


Best Parameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
Best Score: 0.5959472716125076
Accuracy: 0.6057142857142858
[[302  59  11   0]
 [115 170  22   0]
 [ 59  61  47   0]
 [ 11   8  10   0]]
